In [88]:
import gurobipy as gp

def build_instance(file_name):
    print(f"BUILDING INSTANCE {file_name.split("/")[len(file_name.split("/"))-1]}")
    instance_file = open(file_name,"r")
    
    lines = [line.strip() for line in instance_file.readlines()]
    current_line = 0
    
    #Salvando quantidade de clientes e quantidade e lockers
    qty_customer = int(lines[current_line].split(" ")[0])
    qty_locker = int(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    max_vehicle = lines[current_line].split(" ")[0]
    vehicle_capacity = lines[current_line].split(" ")[1]
    
    current_line += 1
    
    demmands = list()
    ## Salvando demmands de clients
    for demmand in range(current_line,qty_customer+2):
        demmands.append(lines[demmand])
    
    current_line += qty_customer
    
    #Preenchendo nos
    nodes = dict()
    
    #Deposito
    depot = {
        'label': "D",
        'x' : float(lines[current_line].split(" ")[0]),
        'y' : float(lines[current_line].split(" ")[1]),
        'earliest' : float(lines[current_line].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[current_line].split(" ")[4]),
        'demand': 0,
        'type' : int(lines[current_line].split(" ")[5])
    }
    nodes['D']=depot
    current_line += 1

    #Clientes
    customers = dict()
    customer_count = 1
    for i in range(current_line, current_line + qty_customer):
        customer = {
        'label' : f"C{customer_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': demmands[customer_count-1],
        'type' : int(lines[i].split(" ")[5])
        }
        customer_count+=1
        customers[customer['label']]=customer
        nodes[customer['label']]=customer
    current_line += qty_customer

    #Lockers
    lockers = dict()
    locker_count = 1
    for i in range(current_line,current_line+qty_locker):
        locker = {
        'label' : f"P{locker_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': 0,
        'customers': list(),
        'type' : int(lines[i].split(" ")[5])
        }
        locker_count+=1
        lockers[locker['label']] = locker
        nodes[locker['label']]=locker
    current_line += qty_locker

    #Atribuição de lockers
    current_customer = 0
    for i in range(current_line, current_line+qty_customer):
        try:
            customer = customers[list(customers.keys())[current_customer]]
            
            locker_index = lines[i].split(" ").index("1")
            locker = lockers[list(lockers.keys())[locker_index]]
            customer['locker'] = locker['label']
            locker['customers'].append(customer['label'])
        except ValueError:    
            pass
        current_customer += 1
    current_line += qty_customer
    
    #Matriz de Distancia
    costs = dict()
    for i,node_i in enumerate(nodes):
         current_node_i = nodes[node_i]
         current_node_i['index'] = i
         for j,node_j in enumerate(nodes):
             current_node_j = nodes[node_j]
             distance = ((current_node_i['x']-current_node_j['x'])**2 + (current_node_i['y']-current_node_j['y'])**2)**(1/2)
             costs[f"c_{current_node_i['label']}{current_node_j['label']}"] = distance
      
    print(f"Qty customer:{qty_customer}")
    print(f"Qty locker:{qty_locker}")
    print(f"Max vehicles:{max_vehicle}")
    print(f"Vehicle capacity:{vehicle_capacity}")
    print(f"BUILD FINISHED")
    return {
        'qty_customer': qty_customer,
        'qty_locker': qty_locker,
        'qty_nodes': qty_locker,
        'max_vehicle': max_vehicle,
        'vehicle_capacity': vehicle_capacity,
        'nodes': nodes,
        'customers': customers,
        'lockers': lockers,
        'depot': depot,
        'costs': costs
    }


instance = build_instance("../../instances/vrppl/inst_test")


BUILDING INSTANCE inst_test
Qty customer:10
Qty locker:2
Max vehicles:2
Vehicle capacity:200
BUILD FINISHED


In [76]:
qty_vehicles = 2

# demands = [node['demand'] for node in instance['nodes']]

#Rotulos

#Nos
nodes = instance['nodes']
node_labels = list(nodes.keys())
customer_labels = list(instance['customers'].keys())
#Veiculos
vehicles = list()
for i in range(qty_vehicles):
    vehicles.append(f"v_{i}")

#Criando dicts

#Custos
costs = instance['costs']

#Criando modelo
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag,0)

#Inserindo variaveis
x = m.addVars(vehicles, node_labels, node_labels, vtype = gp.GRB.BINARY)
# print(x)
#Inserindo restrição
c1 = m.addConstrs(
    gp.quicksum(x[vehicle,node_i,node_j] for vehicle in vehicles for node_j in node_labels) == 1  for node_i in node_labels if int(nodes[node_i]['type']) == 1
)

c3 = m.addConstrs(
    gp.quicksum(x[vehicle,node_i,node_j] for vehicle in vehicles for node_j in node_labels) == 0  for node_i in node_labels if int(nodes[node_i]['type']) == 2
)

c4 = m.addConstrs(
    gp.quicksum(x[vehicle,"D",node_j] for node_j in customer_labels) <= 1 for vehicle in vehicles
)

c5 = m.addConstrs(
    (gp.quicksum(x[vehicle,node_i,node_j] for node_i in node_labels if node_i!=node_j) - gp.quicksum(x[vehicle,node_j,node_i] for node_i in node_labels  if node_i!=node_j)) == 0 for vehicle in vehicles for node_j in node_labels
)

c6 = m.addConstrs(
    gp.quicksum(x[vehicle, node_i, "D"] for node_i in customer_labels) <= 1 for vehicle in vehicles
)


#Definindo função objetivo
m.setObjective(
    gp.quicksum(costs[f"c_{node_i}{node_j}"] * x[vehicle,node_i,node_j] for node_i in node_labels for node_j in node_labels for vehicle in vehicles),
    sense = gp.GRB.MINIMIZE
)
#Executando modelo

m.optimize()

print("Valor Total", m.objVal)
# print(m)
print(m.objVal)
# print(m.getObjective())

for vehicle in vehicles:
    print(vehicle)
    for node_i in node_labels:
        for node_j in node_labels:
            if round(x[vehicle,node_i,node_j].X) == 1:
                print(f"x_{node_i}{node_j}", " ", end="")
    print('\n')
# print(x)
# #Clientes
# rot_customers = [node['label'] for node in instance['customers']]

# #Lockers
# rot_lockers = [node['label'] for node in instance['lockers']]


# print(costs)

# print(vehicles)
# print(edges)
# print(vehicles)
# print(demands)
# print(rot_nodes)
# print(rot_customers)
# print(rot_lockers)



Valor Total 0.0
0.0
v_0
x_DD  x_C4C4  x_C5C5  x_C6C6  x_C7C7  x_C8C8  x_C9C9  x_C10C10  x_P1P1  x_P2P2  

v_1
x_DD  x_C4C4  x_C5C5  x_C6C6  x_C8C8  x_P1P1  x_P2P2  

